## Deliverable 2. Create a Customer Travel Destinations Map.

In [17]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append('..')
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [18]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv('../Weather_Database/WeatherPy_Database.csv')
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Description
0,0,Cherskiy,68.7500,161.3000,20.68,94,100,9.46,RU,2022-05-08 19:39:41,light snow
1,1,Camana,-16.6228,-72.7111,70.47,73,52,10.63,PE,2022-05-08 19:39:42,broken clouds
2,2,Sakakah,29.9697,40.2064,71.17,16,51,7.02,SA,2022-05-08 19:39:42,broken clouds
3,3,Seia,40.4151,-7.7086,62.10,62,6,1.90,PT,2022-05-08 19:39:43,clear sky
4,4,Belmonte,-15.8631,-38.8828,75.92,85,90,10.69,BR,2022-05-08 19:39:43,overcast clouds


In [22]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input('What is the minimum temperature you\'d like for your trip? \U0001F321 '))
max_temp = float(input('What is the maximum temperature you\'d like for your trip? \U0001F321 '))

What is the minimum temperature you'd like for your trip? 🌡 -100
What is the maximum temperature you'd like for your trip? 🌡 31.99


In [23]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[
    (city_data_df['Max Temp'] <= max_temp) 
    & (city_data_df['Max Temp'] >= min_temp)
]

In [24]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID        49
City           49
Lat            49
Lng            49
Max Temp       49
Humidity       49
Cloudiness     49
Wind Speed     49
Country        49
Date           49
Description    49
dtype: int64

In [25]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [26]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[['City', 'Country', 'Max Temp', 'Description', 'Lat', 'Lng']].copy()

# 5b. Create a new column 'Hotel Name'
hotel_df['Hotel Name'] = ''
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Cherskiy,RU,20.68,light snow,68.7500,161.3000,
6,Vanavara,RU,25.97,overcast clouds,60.3400,102.2797,
16,Talaya,RU,22.57,scattered clouds,61.3833,152.7500,
19,Dukat,RU,23.25,clear sky,62.5500,155.5500,
42,Yellowknife,CA,28.92,light snow,62.4560,-114.3525,
47,Mezen,RU,30.69,overcast clouds,65.8522,44.2400,
49,Baykit,RU,22.48,overcast clouds,61.6700,96.3700,
73,Yerbogachen,RU,27.84,light snow,61.2767,108.0108,
87,Khatanga,RU,2.59,broken clouds,71.9667,102.5000,
89,Lesosibirsk,RU,27.12,few clouds,58.2358,92.4828,


In [27]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    'radius': 5000,
    'type': 'lodging',
    'key': g_key
}
# 6d. Set up the base URL for the Google Directions API to get JSON data.
base_url = 'https://maps.googleapis.com/maps/api/place/nearbysearch/json'

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']

    # 6e. Make request and retrieve the JSON data from the search. 
    params['location'] = f'{lat},{lng}'
    hotels = requests.get(base_url, params=params).json()

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, 'Hotel Name'] = hotels['results'][0]['name']
    except IndexError:
        print('Hotel not found.. skipping!')

Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!
Hotel not found.. skipping!


In [28]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df['Hotel Name'] != '')]
clean_hotel_df.head()

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
0,Cherskiy,RU,20.68,light snow,68.7500,161.3000,Gostinitsa
42,Yellowknife,CA,28.92,light snow,62.4560,-114.3525,The Explorer Hotel
47,Mezen,RU,30.69,overcast clouds,65.8522,44.2400,Gostevoy Dom Everest
87,Khatanga,RU,2.59,broken clouds,71.9667,102.5000,"""Khatanga"""
89,Lesosibirsk,RU,27.12,few clouds,58.2358,92.4828,"""Vitalina"" hotel"


In [29]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label='City_ID')

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = '''
<dl>
<dt><strong>Hotel Name</strong></dt><dd>{Hotel Name}</dd>
<dt><strong>City</strong></dt><dd>{City}</dd>
<dt><strong>Country</strong></dt><dd>{Country}</dd>
<dt><strong>Max Temp</strong></dt><dd>{Max Temp} °F</dd>
<dt><strong>Description</strong></dt><dd>{Description}</dd>
</dl>
'''

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[['Lat', 'Lng']]

In [35]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig 

Figure(layout=FigureLayout(height='420px'))